In [ ]:
mytask='all'        #'diagnostic', 'subdiagnostic', 'superdiagnostic', 'form', 'rhythm', 'all'

import os
os.environ["CUDA_VISIBLE_DEVICES"] ='1'
device='cuda:0'

database_csv='./ptbxl/ptbxl_database.csv'
raw100_npy='./ptbxl/raw100.npy'
ptbxl_folder='./ptbxl/'

import pickle

import pandas as pd
import numpy as np
import ast

X = pd.read_csv(database_csv, index_col='ecg_id')

ZZ = np.zeros((21837,4))
ZZ [:,0]=np.array(X['age'])
ZZ [:,1]=np.array(X['sex'])
ZZ [:,2]=np.array(X['height'])
ZZ [:,3]=np.array(X['weight'])
ZZ [np.isnan(ZZ)]=300  
ZZ [ZZ==300]=-100
print(type(ZZ))
print(ZZ)
print(ZZ.shape)
ZZ [:,0]=ZZ [:,0]/100 
ZZ [:,2]=ZZ [:,2]/200
ZZ [:,3]=ZZ [:,3]/100

print(ZZ)


Y = pd.read_csv(database_csv, index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
X = np.load(raw100_npy, allow_pickle=True)

def compute_label_aggregations(df, folder=ptbxl_folder, ctype='superdiagnostic'):

    df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))

    aggregation_df = pd.read_csv(folder+'scp_statements.csv', index_col=0)

    if ctype in ['diagnostic', 'subdiagnostic', 'superdiagnostic']:

        def aggregate_all_diagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    tmp.append(key)
            return list(set(tmp))

        def aggregate_subdiagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    c = diag_agg_df.loc[key].diagnostic_subclass
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        def aggregate_diagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    c = diag_agg_df.loc[key].diagnostic_class
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        diag_agg_df = aggregation_df[aggregation_df.diagnostic == 1.0]
        if ctype == 'diagnostic':
            df['diagnostic'] = df.scp_codes.apply(aggregate_all_diagnostic)
            df['diagnostic_len'] = df.diagnostic.apply(lambda x: len(x))
        elif ctype == 'subdiagnostic':
            df['subdiagnostic'] = df.scp_codes.apply(aggregate_subdiagnostic)
            df['subdiagnostic_len'] = df.subdiagnostic.apply(lambda x: len(x))
        elif ctype == 'superdiagnostic':
            df['superdiagnostic'] = df.scp_codes.apply(aggregate_diagnostic)
            df['superdiagnostic_len'] = df.superdiagnostic.apply(lambda x: len(x))
    elif ctype == 'form':
        form_agg_df = aggregation_df[aggregation_df.form == 1.0]

        def aggregate_form(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in form_agg_df.index:
                    c = key
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        df['form'] = df.scp_codes.apply(aggregate_form)
        df['form_len'] = df.form.apply(lambda x: len(x))
    elif ctype == 'rhythm':
        rhythm_agg_df = aggregation_df[aggregation_df.rhythm == 1.0]

        def aggregate_rhythm(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in rhythm_agg_df.index:
                    c = key
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        df['rhythm'] = df.scp_codes.apply(aggregate_rhythm)
        df['rhythm_len'] = df.rhythm.apply(lambda x: len(x))
    elif ctype == 'all':
        df['all_scp'] = df.scp_codes.apply(lambda x: list(set(x.keys())))

    return df

def select_data_with_meta(XX,YY,ZZ, ctype, min_samples, outputfolder):
    # convert multilabel to multi-hot
    mlb = MultiLabelBinarizer()

    if ctype == 'diagnostic':
        X = XX[YY.diagnostic_len > 0]
        Y = YY[YY.diagnostic_len > 0]
        Z = ZZ[YY.diagnostic_len > 0]
        mlb.fit(Y.diagnostic.values)
        y = mlb.transform(Y.diagnostic.values)
    elif ctype == 'subdiagnostic':
        counts = pd.Series(np.concatenate(YY.subdiagnostic.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.subdiagnostic = YY.subdiagnostic.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['subdiagnostic_len'] = YY.subdiagnostic.apply(lambda x: len(x))
        X = XX[YY.subdiagnostic_len > 0]
        Y = YY[YY.subdiagnostic_len > 0]
        Z = ZZ[YY.subdiagnostic_len > 0]
        mlb.fit(Y.subdiagnostic.values)
        y = mlb.transform(Y.subdiagnostic.values)
    elif ctype == 'superdiagnostic':
        counts = pd.Series(np.concatenate(YY.superdiagnostic.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.superdiagnostic = YY.superdiagnostic.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['superdiagnostic_len'] = YY.superdiagnostic.apply(lambda x: len(x))
        X = XX[YY.superdiagnostic_len > 0]
        Y = YY[YY.superdiagnostic_len > 0]
        Z = ZZ[YY.superdiagnostic_len > 0]
        mlb.fit(Y.superdiagnostic.values)
        y = mlb.transform(Y.superdiagnostic.values)
    elif ctype == 'form':
        # filter
        counts = pd.Series(np.concatenate(YY.form.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.form = YY.form.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['form_len'] = YY.form.apply(lambda x: len(x))
        # select
        X = XX[YY.form_len > 0]
        Y = YY[YY.form_len > 0]
        Z = ZZ[YY.form_len > 0]
        mlb.fit(Y.form.values)
        y = mlb.transform(Y.form.values)
    elif ctype == 'rhythm':
        # filter 
        counts = pd.Series(np.concatenate(YY.rhythm.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.rhythm = YY.rhythm.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['rhythm_len'] = YY.rhythm.apply(lambda x: len(x))
        # select
        X = XX[YY.rhythm_len > 0]
        Y = YY[YY.rhythm_len > 0]
        Z = ZZ[YY.rhythm_len > 0]
        mlb.fit(Y.rhythm.values)
        y = mlb.transform(Y.rhythm.values)
    elif ctype == 'all':
        # filter 
        counts = pd.Series(np.concatenate(YY.all_scp.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.all_scp = YY.all_scp.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['all_scp_len'] = YY.all_scp.apply(lambda x: len(x))
        # select
        X = XX[YY.all_scp_len > 0]
        Y = YY[YY.all_scp_len > 0]
        Z = ZZ[YY.all_scp_len > 0]
        mlb.fit(Y.all_scp.values)
        y = mlb.transform(Y.all_scp.values)
    else:
        pass

    return X, Y, Z, y, mlb

labels = compute_label_aggregations(Y, ptbxl_folder, mytask)

def preprocess_signals(X_train, X_validation, X_test, outputfolder):
    # Standardize data such that mean 0 and variance 1
    ss = StandardScaler()
    ss.fit(np.vstack(X_train).flatten()[:,np.newaxis].astype(float))

    return apply_standardizer(X_train, ss), apply_standardizer(X_validation, ss), apply_standardizer(X_test, ss)

def apply_standardizer(X, ss):
    X_tmp = []
    for x in X:
        x_shape = x.shape
        X_tmp.append(ss.transform(x.flatten()[:,np.newaxis]).reshape(x_shape))
    X_tmp = np.array(X_tmp)
    return X_tmp
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
# Select relevant data and convert to one-hot
data,labels,Z,Y, _ = select_data_with_meta(X, labels, ZZ, mytask, 0, '.')
print('ptbxl labels shape is:',labels.shape)
print('n_classes number shape is:',Y.shape)
print(Y[0,:])
input_shape = data[0].shape

# 10th fold for testing (9th for now)
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]
z_test = Z[labels.strat_fold == 10]
# 9th fold for validation (8th for now)
X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]
z_val = Z[labels.strat_fold == 9]
# rest for training
X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]
z_train = Z[labels.strat_fold <= 8]

# Preprocess signal data
X_train, X_val, X_test = preprocess_signals(X_train, X_val, X_test, '.')
n_classes = y_train.shape[1]
print(y_train.shape)
print(X_train.shape) # 17441,1000,12
print(z_train.shape) # 17441,4

# 把z扩充为17441，4，12，然后和X在第二维融合在一起

z_train=np.expand_dims(z_train,axis=2).repeat(12,axis=2)
z_val=np.expand_dims(z_val,axis=2).repeat(12,axis=2)
z_test=np.expand_dims(z_test,axis=2).repeat(12,axis=2)

X_train=np.concatenate((X_train,z_train),axis=1)
X_val=np.concatenate((X_val,z_val),axis=1)
X_test=np.concatenate((X_test,z_test),axis=1)

print(X_train.shape)

import itertools
from itertools import repeat
import shutil
import datetime as dtm
import time
from scipy.signal import stft
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random
from numpy import concatenate
from collections import OrderedDict
from torchsummary import summary
import math
import torch.optim as optim
import torch.utils.data as Data
import timm
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model
import sklearn 
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,f1_score
from scipy import signal
from numpy import mean
from torch.nn.modules import conv
from torch.nn.modules.utils import _pair
from torch.optim.optimizer import Optimizer, required
from torch.autograd import Variable
from torch import Tensor
from torch.nn import Parameter
from math import ceil
from torch import nn, einsum
from enum import Enum
import re
import inspect
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
print('import ok')

def get_np_posemb(patch_nums,emb_dim,cls_token=False):
    out_np_pe=np.zeros((patch_nums,emb_dim))
    for tmp_emb_index in range(emb_dim):
        tmp_emb_k=tmp_emb_index//2
        tmp_emb_omega=10000**(2*tmp_emb_k/emb_dim)
        if tmp_emb_index%2==0:            
            for tmp_patch_index in range(patch_nums):
                out_np_pe[tmp_patch_index,tmp_emb_index]=np.sin(float(tmp_patch_index)/tmp_emb_omega)
        elif tmp_emb_index%2==1:            
            for tmp_patch_index in range(patch_nums):
                out_np_pe[tmp_patch_index,tmp_emb_index]=np.cos(float(tmp_patch_index)/tmp_emb_omega)
        else:
            print('error pos emb!')
    if cls_token:
        out_np_pe=np.concatenate([np.zeros([1, emb_dim]), out_np_pe], axis=0)
    return out_np_pe
            
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)  
        self.fn = fn  
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Temporal_Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads  
        project_out = not (heads == 1 and dim_head == dim)  
        self.heads = heads  
        self.scale = dim_head ** -0.5  
        self.attend = nn.Softmax(dim = -1) 
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)  
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()
        
    def forward(self, x):  
        
        x=rearrange(x, 'b (tws twn) c -> (b tws) twn c', tws = 6)
        b, n, _, h = *x.shape, self.heads  
        qkv = self.to_qkv(x).chunk(3, dim = -1)  
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)  
        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale  
        attn = self.attend(dots)  
        out = einsum('b h i j, b h j d -> b h i d', attn, v)  
        out = rearrange(out, 'b h n d -> b n (h d)')  
        out=rearrange(out, '(b tws) twn c -> b (tws twn) c', tws = 6)
        final_out=self.to_out(out)
        return final_out  
    
class Leadview_Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads 
        project_out = not (heads == 1 and dim_head == dim)  
        self.heads = heads 
        self.scale = dim_head ** -0.5  
        self.attend = nn.Softmax(dim = -1)  
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)  
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()
        
    def forward(self, x): 
        
        
        x=rearrange(x, 'b (lgn lgs) c-> (b lgs) lgn c', lgs = 12)
        b, n, _, h = *x.shape, self.heads  
        qkv = self.to_qkv(x).chunk(3, dim = -1)  
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)  
        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale  
        attn = self.attend(dots)  
        out = einsum('b h i j, b h j d -> b h i d', attn, v)  
        out = rearrange(out, 'b h n d -> b n (h d)') 
        out=rearrange(out, '(b lgs) lgn c -> b (lgn lgs) c', lgs = 12)
        final_out=self.to_out(out)
        return final_out      
    

class F_Attention(nn.Module):  
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads  
        project_out = not (heads == 1 and dim_head == dim) 
        self.heads = heads 
        self.scale = dim_head ** -0.5  
        self.attend = nn.Softmax(dim = -1) 
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False) 
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()
        
    def forward(self, x):
        
        b, n, _, h = *x.shape, self.heads 
        qkv = self.to_qkv(x).chunk(3, dim = -1) 
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)  
        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale 
        attn = self.attend(dots) 
        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)') 
        final_out=self.to_out(out)
        return final_out 
    
    
class Da_Transformer(nn.Module):
    def __init__(self, dim, depth, mlp_dim, dropout = 0. ):
        super().__init__()
        heads=5
        dim_head=dim//heads
        self.layers = nn.ModuleList([])  
        self.aug_shortcut=nn.Sequential(
                nn.Linear(dim,dim),
                nn.GELU(),
        )
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Temporal_Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),  
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout)),  
                PreNorm(dim, Leadview_Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)), 
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout)),  
            ]))
    def forward(self, x):
        for t_attn, t_ff, l_attn, l_ff in self.layers:
            x = t_attn(x) + x 
            x = t_ff(x) + x        
            x = l_attn(x) + x 
            x = l_ff(x) + x   
        return x

class F_Transformer(nn.Module):
    def __init__(self, dim, depth, mlp_dim, dropout = 0. ):
        super().__init__()
        heads=5
        dim_head=dim//heads
        self.layers = nn.ModuleList([]) 
        self.aug_shortcut=nn.Sequential(
                nn.Linear(dim,dim),
                nn.GELU(),
        )
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, F_Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)), 
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout)) 
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x 
            x = ff(x) + x        
        return x    
    
class ECGViT(nn.Module):
    def __init__(self, *, signal_length: int=250, patch_size:int=50, num_classes:int=71, dim:int=1024, Da_depth:int=4, F_depth:int=2, mlp_dim:int=2048, channels:int = 1,leads:int=12, dropout = 0.,meta_info=0):
        super().__init__()
        assert signal_length % patch_size == 0, 'signal_length must be divisible by the patch size.' 
        self.leads=leads
        self.patch_size=patch_size
        self.meta_info=meta_info
        num_patches = self.leads*(signal_length  // patch_size)+self.leads  
        patch_dim = channels * patch_size *1  

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = 1, p2 = patch_size), 
            nn.Linear(patch_dim, dim),  
        )

        self.pos_emb_1 = nn.Parameter(torch.randn(1,num_patches,dim),requires_grad=False)
        self.np_posemb_1=get_np_posemb(num_patches,self.pos_emb_1.shape[2],cls_token=False)
        self.pos_emb_1.data.copy_(torch.from_numpy(self.np_posemb_1).float().unsqueeze(0))
        
        self.pos_emb_2 = nn.Parameter(torch.randn(1,num_patches+self.meta_info+1,dim),requires_grad=False)
        self.np_posemb_2=get_np_posemb(num_patches+self.meta_info,self.pos_emb_2.shape[2],cls_token=True)
        self.pos_emb_2.data.copy_(torch.from_numpy(self.np_posemb_2).float().unsqueeze(0))
        
        self.Da_transformer = Da_Transformer(dim, Da_depth,mlp_dim, dropout) 
        self.F_transformer = F_Transformer(dim, F_depth,mlp_dim, dropout) 
        
        self.to_latent = nn.Identity()  

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim), 
            nn.Linear(dim, num_classes) 
        )
        self.maxpool_global=nn.MaxPool2d((1, 5), stride=(1, 5))
    
        self.meta_to_embedding=nn.Linear(self.leads,dim)
        
    def forward(self, insig): # bs,12,254
        sig=insig[:,:,:-1*self.meta_info]
        meta_sig=insig[:,:,-1*self.meta_info:].transpose(-1,-2)  # bs,4,12
        sig1=sig.unsqueeze(1)        
        pooled_sig=torch.zeros((sig1.shape[0],sig1.shape[1],sig1.shape[2],sig1.shape[3]+self.patch_size)).to(device)
        pooled_sig[:,:,:,:sig1.shape[3]]=sig1
        pooled_sig[:,:,:,sig1.shape[3]:sig1.shape[3]+self.patch_size]=self.maxpool_global(sig1)
        x = self.to_patch_embedding(pooled_sig) 
        b, n, _ = x.shape  
        x2 = x + self.pos_emb_1 
        x3 = self.Da_transformer(x2)
        meta_tokens=self.meta_to_embedding(meta_sig)
        cls_tokens =  x3.mean(dim = 1).unsqueeze(1) 
        x4 = torch.cat((cls_tokens,meta_tokens,x3), dim=1)
        x4_=x4+self.pos_emb_2
        x5 = self.F_transformer(x4_) 
        x6 = x5[:, 0]    
        x7 = self.to_latent(x6)
        x8 = self.mlp_head(x7)
        return x8

ECGViT_model=ECGViT(num_classes = 71,dim = 160,mlp_dim = 480,Da_depth=4, F_depth=2,patch_size=50,leads=12,dropout=0.2,meta_info=4).to(device) 


def train_one_epoch(epoch, model,loss_fn, optimizer, train_loader, device, scheduler=None,warmup=False):
    global ema
    model.train() 
    t = time.time()
    running_loss = None 
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (in_data_, targets_) in pbar:
        data_=in_data_[:,:1000,:].transpose(-1,-2).type(torch.FloatTensor).to(device)
        meta_=in_data_[:,1000:,:].transpose(-1,-2).type(torch.FloatTensor).to(device)
        
        optimizer.zero_grad()
        for tmp_seg in range(ceil((data_.shape[-1])/125-0.1)-1):
            if (tmp_seg==0):
                random_bias=random.randint(0,int(125/2))
            elif (tmp_seg==(ceil((data_.shape[-1])/125-0.1)-2)):
                random_bias=random.randint(-int(125/2),0)
            else:
                random_bias=random.randint(-int(125/2),int(125/2))

            seg_data_=torch.cat((data_[:,:,125*tmp_seg+random_bias:125*(tmp_seg+2)+random_bias],meta_),-1)
            preds_=model(seg_data_).type(torch.cuda.FloatTensor)
            targets_=targets_.type(torch.FloatTensor).to(device)
            loss=loss_fn(preds_,targets_)
            loss.backward()
            optimizer.first_step(zero_grad=True)
            preds_=model(seg_data_).type(torch.cuda.FloatTensor)
            targets_=targets_.type(torch.FloatTensor).to(device)
            loss=loss_fn(preds_,targets_)
            loss.backward()
            optimizer.second_step(zero_grad=True)
        if running_loss is None:
            running_loss = loss.item() 
        else:
            running_loss = running_loss * .99 + loss.item() * .01           
        description = f'epoch {epoch} loss: {running_loss:.4f}'
        pbar.set_description(description)
        ema.update()

    if warmup==True:
        if scheduler!=None:
            scheduler.step()   
    return running_loss
print('ok')


def val_one_epoch_wEMA(epoch, model,loss_fn,  val_loader, device,scheduler):
    global ema
    ema.apply_shadow()
    model.eval() 
    val_tar=0
    mean_val_pred=0
    max_val_pred=0
    with torch.no_grad():
        t = time.time()
        running_loss = None 
        pbar = tqdm(enumerate(val_loader), total=len(val_loader))
        for step, (in_data_, targets_) in pbar:
            data_=in_data_[:,:1000,:].transpose(-1,-2).type(torch.FloatTensor).to(device)
            meta_=in_data_[:,1000:,:].transpose(-1,-2).type(torch.FloatTensor).to(device)
            all_seg_preds_=torch.zeros((ceil((data_.shape[-1])/125-0.1)-1,targets_.shape[0],targets_.shape[1])).type(torch.FloatTensor).to(device) 
            for tmp_seg in range(ceil((data_.shape[-1])/125-0.1)-1):
                seg_data_=torch.cat((data_[:,:,125*tmp_seg:125*(tmp_seg+2)],meta_),-1)
                seg_preds_=model(seg_data_).type(torch.cuda.FloatTensor)
                all_seg_preds_[tmp_seg,:,:]=seg_preds_.clone().detach().type(torch.cuda.FloatTensor) 
            agg_preds_=(all_seg_preds_.mean(dim=0)).type(torch.FloatTensor).to(device)
            targets_=targets_.type(torch.FloatTensor).to(device) 
            loss=loss_fn(agg_preds_,targets_)
            if running_loss is None:
                running_loss_list=[]
                running_loss = loss.item()
                running_loss_list.append(loss.item()) 
            else:
                running_loss = running_loss * .99 + loss.item() * .01 
                running_loss_list.append(loss.item()) 
            description = f'epoch {epoch} mean loss: {mean(running_loss_list):.4f}' 
            pbar.set_description(description)  
            all_seg_preds_=torch.sigmoid(all_seg_preds_)
            if type(mean_val_pred)==int:
                mean_val_pred=(all_seg_preds_.mean(dim=0)).type(torch.FloatTensor).clone().detach().cpu().numpy()
            else:
                mean_val_pred=np.concatenate((mean_val_pred,
                                    (all_seg_preds_.mean(dim=0)).type(torch.FloatTensor).clone().detach().cpu().numpy(),
                                    ),axis=0)
            if type(val_tar)==int:
                val_tar=targets_.clone().detach().cpu().numpy()
            else:
                val_tar=np.concatenate((val_tar,
                                    targets_.clone().detach().cpu().numpy(),
                                    ),axis=0)
    mean_macro_auc=roc_auc_score(val_tar, mean_val_pred, average='macro')
    print('mean_macro_auc= ',mean_macro_auc)
    if scheduler!=None:
        scheduler.step(mean_macro_auc)    
    ema.restore()
    return mean_macro_auc
print('ok')

def test_one_epoch_wEMA(epoch, model,loss_fn,test_loader, device):
    global ema
    ema.apply_shadow()
    model.eval() 
    test_tar=0
    mean_test_pred=0
    max_test_pred=0
    with torch.no_grad():
        t = time.time()
        running_loss = None 
        pbar = tqdm(enumerate(test_loader), total=len(test_loader))
        for step, (in_data_, targets_) in pbar:#bs,1000，12    bs,44
            data_=in_data_[:,:1000,:].transpose(-1,-2).type(torch.FloatTensor).to(device)#12，1000
            meta_=in_data_[:,1000:,:].transpose(-1,-2).type(torch.FloatTensor).to(device)#12，4
            all_seg_preds_=torch.zeros((ceil((data_.shape[-1])/125-0.1)-1,targets_.shape[0],targets_.shape[1])).type(torch.FloatTensor).to(device)  #7，bs，44
            for tmp_seg in range(ceil((data_.shape[-1])/125-0.1)-1):
                seg_data_=torch.cat((data_[:,:,125*tmp_seg:125*(tmp_seg+2)],meta_),-1)        
                seg_preds_=model(seg_data_).type(torch.cuda.FloatTensor)# bs,44
                all_seg_preds_[tmp_seg,:,:]=seg_preds_.clone().detach().type(torch.cuda.FloatTensor) # bs,44
            agg_preds_=(all_seg_preds_.mean(dim=0)).type(torch.FloatTensor).to(device)
            targets_=targets_.type(torch.FloatTensor).to(device) # bs,1000,12
            loss=loss_fn(agg_preds_,targets_)
            if running_loss is None:
                running_loss_list=[]
                running_loss = loss.item()
                running_loss_list.append(loss.item()) 
            else:
                running_loss = running_loss * .99 + loss.item() * .01 
                running_loss_list.append(loss.item()) 
            description = f'epoch {epoch} mean loss: {mean(running_loss_list):.4f}' 
            pbar.set_description(description)  
            all_seg_preds_=torch.sigmoid(all_seg_preds_)
            if type(mean_test_pred)==int:
                mean_test_pred=(all_seg_preds_.mean(dim=0)).type(torch.FloatTensor).clone().detach().cpu().numpy()
            else:
                mean_test_pred=np.concatenate((mean_test_pred,
                                    (all_seg_preds_.mean(dim=0)).type(torch.FloatTensor).clone().detach().cpu().numpy(),
                                    ),axis=0)
            if type(test_tar)==int:
                test_tar=targets_.clone().detach().cpu().numpy()
            else:
                test_tar=np.concatenate((test_tar,
                                    targets_.clone().detach().cpu().numpy(),
                                    ),axis=0)
    mean_macro_auc=roc_auc_score(test_tar, mean_test_pred, average='macro')
    print('mean_macro_auc= ',mean_macro_auc)
    ema.restore()
    return mean_macro_auc
print('ok')


class GetLoader(torch.utils.data.Dataset):
    def __init__(self, data_data, data_label):
        self.data = data_data
        self.label = data_label
    def __getitem__(self, index):
        data=self.data[index]
        labels=self.label[index]
        return data, labels
    def __len__(self):
        return len(self.data)
    
origin_data_train=X_train
label_train=y_train
origin_data_val=X_val
label_val=y_val
origin_data_test=X_test
label_test=y_test

print('load ok')

data_train=origin_data_train
data_val=origin_data_val
data_test=origin_data_test

if mytask=='form':
    # 'The task has a smaller amount of data. To achieve the unification of the process, we copy the data to obtain similar training data.
    data_train=np.concatenate((data_train,data_train),axis=0)
    label_train=np.concatenate((label_train,label_train),axis=0)

print('data_val.shape=',data_val.shape)
print('data_test.shape=',data_test.shape)
print('data_train.shape=',data_train.shape)
print('label_train.shape=',label_train.shape)
print('label_val.shape=',label_val.shape)
print('label_test.shape=',label_test.shape)
bs1=128
bs2=128
bs3=128

train_datasloader = DataLoader(GetLoader(data_train, label_train), bs1, shuffle=True, drop_last=False, num_workers=2)
val_datasloader = DataLoader(GetLoader(data_val, label_val), bs2, shuffle=False, drop_last=False, num_workers=2)
test_datasloader = DataLoader(GetLoader(data_test, label_test), bs3, shuffle=False, drop_last=False, num_workers=2)

#EMA
class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}
        #self.waive_epoch=waive_epoch
        self.count=0
        
    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                tmp_decay=self.decay
                self.count+=1    
                new_average = (1.0 - tmp_decay) * param.data + tmp_decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}


class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()


    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm
    
    def adjust_lr_to(self,a:float):
        self.param_groups[0]['lr']=a
    

ECGViT_model=ECGViT(num_classes = label_train.shape[-1],dim = 160,mlp_dim = 480,Da_depth=4, F_depth=2,patch_size=50,leads=12,dropout=0.2,meta_info=4).to(device) 

global ema
train_loss = []  
val_mean_loss = []
test_macro_auc=[]
ema = EMA(ECGViT_model, 0.998)#, waive_epoch=args['warm_epoch'])
ema.register()
loss_fn=F.binary_cross_entropy_with_logits
recommand_lr=3.E-3
print('recommand_lr=',recommand_lr)

# warmup
warm_epoch=20
total_epoch=60
beta2=0.99
finetune_weight_decay=0.05
base_optimizer=torch.optim.AdamW
optimizer = SAM(ECGViT_model.parameters(),base_optimizer,lr=recommand_lr, betas=(0.9, beta2), eps=1e-08, weight_decay=finetune_weight_decay)
scheduler = None   
for epoch in range (warm_epoch):
    print('current lr=',recommand_lr*(epoch+1)/warm_epoch)
    optimizer.adjust_lr_to(recommand_lr*(epoch+1)/warm_epoch)
    t_loss=train_one_epoch(epoch, ECGViT_model, loss_fn, optimizer, train_datasloader, device,scheduler,warmup=True)
    torch.cuda.empty_cache()  

# train

scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=0,min_lr=0, threshold=1e-7, threshold_mode='rel',eps=1e-5, verbose=True)

best_val_w=0.
best_epoch_o=0
final_test_w=0.
best_val_o=0.
best_epoch_o=0
final_test_o=0.
conlow=0

for epoch in range(warm_epoch,total_epoch):
    print('_____________________________________')
    print(' ')
    t_loss=train_one_epoch(epoch, ECGViT_model, loss_fn, optimizer, train_datasloader, device)
    train_loss.append(t_loss)
    wEMA_val_AUC=val_one_epoch_wEMA(epoch, ECGViT_model, loss_fn,val_datasloader, device,scheduler) 
    wEMA_test_AUC=test_one_epoch_wEMA(epoch, ECGViT_model, loss_fn,test_datasloader, device)   
    if (best_val_w<wEMA_val_AUC):
        conlow=0
        best_val_w=wEMA_val_AUC
        best_epoch_w=epoch
        final_test_w=wEMA_test_AUC
        print('find better model w') 
    else:
        conlow+=1
    print('wEMA_val_AUC=',wEMA_val_AUC,'   --')
    print('wEMA_test_AUC=',wEMA_test_AUC,'   --')
    torch.cuda.empty_cache()    
    if conlow==3:
        print('The score of validation set is not improved in three consecutive epochs, triggering the early stopping condition.')
        break
print('found best model w in epoch: ',best_epoch_w)    
print('best_val_w=',best_val_w)
print('final_test_w=',final_test_w)
